# ChatGLM3微调实战-LoRA技术微调

## 步骤1 导入相关包

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# 配置环境变量
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# os.environ['HF_HOME'] = '/root/autodl-tmp/cache/huggingface'
# os.environ['MODELSCOPE_CACHE']='/root/autodl-tmp/cache/modelscope'

# 模型下载
# from modelscope import snapshot_download
# model_dir = snapshot_download('ZhipuAI/chatglm3-6b-base')

# 设置关键参数
dataset_name_or_path = "datasets/identity_zh.json"
model_name_or_path = "models/ZhipuAI/chatglm3-6b-base"
train_output_dir = "saves/lora/chatglm3-6b-base"

## 步骤2 加载数据集

In [2]:
df = pd.read_json(dataset_name_or_path)
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 91
})

In [3]:
ds[:1]

{'instruction': ['hi'],
 'input': [''],
 'output': ['Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?']}

## 步骤3 数据集预处理

In [4]:
#加载本地模型，提前下载到本地
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer

ChatGLMTokenizer(name_or_path='models/ZhipuAI/chatglm3-6b-base', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	
}

补充说明：
- chatglm3-6b 采用的Prompt Template格式
``` text
[gMASK]<sop><|system|>\nSYSTEM_PROMPT<|user|>\nHUMAN_PROMPT<|assistant|>\nAI_OUT<|endoftext|>
```
- 例如
```text
[gMASK]<sop><|system|> 
假设你是皇帝身边的女人--甄嬛。<|user|> 
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|assistant|> 
嘘——都说许愿说破是不灵的。<|endoftext|>
```

In [5]:
def process_func(example):
    MAX_LENGTH = 256 # 设置最大长度为256
    input_ids, attention_mask, labels = [], [], [] # 初始化输入ID、注意力掩码和标签列表
    instruction = "\n".join([example["instruction"], example["input"]])     # prompt
    instruction = tokenizer.build_chat_input(instruction, history=[], role="user")  # [gMASK]sop<|user|> \n prompt <|assistant|>
    response = tokenizer(example["output"], add_special_tokens=False)        # \n response
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id] #eos token
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 91
})

In [7]:
# 检查input_ids的格式
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK]sop<|user|> \n hello\n<|assistant|> Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?'

In [8]:
# 检查labels数据格式
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?'

## 步骤4 创建模型

In [9]:
import torch
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, 
                                             low_cpu_mem_usage=True,
                                             torch_dtype=torch.half,
                                             device_map="auto",
                                             trust_remote_code=True)
model

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

In [10]:
 # 开启梯度检查点时，要执行该方法
# model.enable_input_require_grads()

In [11]:
#  查看数据类型
model.dtype  

torch.float16

In [12]:
# 查看模型参数
for name, param in model.named_parameters():
    print(name)

transformer.embedding.word_embeddings.weight
transformer.encoder.layers.0.input_layernorm.weight
transformer.encoder.layers.0.self_attention.query_key_value.weight
transformer.encoder.layers.0.self_attention.query_key_value.bias
transformer.encoder.layers.0.self_attention.dense.weight
transformer.encoder.layers.0.post_attention_layernorm.weight
transformer.encoder.layers.0.mlp.dense_h_to_4h.weight
transformer.encoder.layers.0.mlp.dense_4h_to_h.weight
transformer.encoder.layers.1.input_layernorm.weight
transformer.encoder.layers.1.self_attention.query_key_value.weight
transformer.encoder.layers.1.self_attention.query_key_value.bias
transformer.encoder.layers.1.self_attention.dense.weight
transformer.encoder.layers.1.post_attention_layernorm.weight
transformer.encoder.layers.1.mlp.dense_h_to_4h.weight
transformer.encoder.layers.1.mlp.dense_4h_to_h.weight
transformer.encoder.layers.2.input_layernorm.weight
transformer.encoder.layers.2.self_attention.query_key_value.weight
transformer.enco

### 1、PEFT 步骤1 配置文件

In [13]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value"], 
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

### 2、PEFT 步骤2 创建模型

In [14]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear(
                  (base_layer): Linear(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4608, bias=False)
          

In [15]:
# 打印出模型中可训练参数的数量
model.print_trainable_parameters()

trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.0312


In [16]:
# 查看模型参数，查看LoRA层添加到哪
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

base_model.model.transformer.embedding.word_embeddings.weight torch.Size([65024, 4096]) torch.float16
base_model.model.transformer.encoder.layers.0.input_layernorm.weight torch.Size([4096]) torch.float16
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.base_layer.weight torch.Size([4608, 4096]) torch.float16
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.base_layer.bias torch.Size([4608]) torch.float16
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.lora_A.default.weight torch.Size([8, 4096]) torch.float32
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.lora_B.default.weight torch.Size([4608, 8]) torch.float32
base_model.model.transformer.encoder.layers.0.self_attention.dense.weight torch.Size([4096, 4096]) torch.float16
base_model.model.transformer.encoder.layers.0.post_attention_layernorm.weight torch.Size([4096]) torch.float16
base_model.model.transformer.encoder.layer

## 步骤5 配置训练参数

In [17]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/chatglm2output", #输出目录，用于存储模型和日志文件。
    per_device_train_batch_size=1, # 每个设备的训练批次大小，即每个设备每次处理的数据量，批次越大，训练时需要资源越多
    gradient_accumulation_steps=8, # 指定梯度累积步数。用于控制梯度更新的频率。在每个累积步中，模型会计算多个批次的梯度，然后一次性更新权重。这可以减少内存占用并提高训练速度。在本例子中，每16个步骤进行一次梯度更新。
    logging_steps=10, #日志记录步数，用于控制每隔多少步记录一次训练日志。
    save_steps=50,  # 每隔多少步保存一次模型
    num_train_epochs=20, #训练轮数，即模型在整个训练集上进行迭代的次数。正常情况会训练很多轮
    learning_rate=1e-4, #学习率，控制模型参数更新的速度。较小的学习率会使模型收敛得更快，但可能需要更多的训练轮数
    # gradient_checkpointing=True,  # 是否启用梯度检查点（减少内存占用）
    adam_epsilon=1e-4, #Adam优化器的epsilon值，用于防止除以零的情况。
    remove_unused_columns=False #是否移除未使用的列，如果设置为False，则保留所有列，否则只保留模型所需的列。
)

## 步骤6 创建训练器

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤7 模型训练

In [19]:
trainer.train()

Step,Training Loss
10,3.303000
20,3.059000
30,2.545600
40,2.029700
50,1.458000
60,1.248400
70,1.124400
80,0.957500
90,0.873500
100,0.819500


TrainOutput(global_step=220, training_loss=1.120823376828974, metrics={'train_runtime': 364.1551, 'train_samples_per_second': 4.998, 'train_steps_per_second': 0.604, 'total_flos': 2599084486557696.0, 'train_loss': 1.120823376828974, 'epoch': 19.34065934065934})

## 步骤8 模型保存

In [20]:
# 保存模型及训练状态
model.save_pretrained(train_output_dir)

## 步骤9 模型推理
- <span style="background-color: yellow;">需要重启并切换jupyter内核</span>

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import PeftModel

# 设置关键参数
model_name_or_path = "models/ZhipuAI/chatglm3-6b-base"
train_output_dir = "saves/lora/chatglm3-6b-base"

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

# 加载Base模型
base_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                  low_cpu_mem_usage=True,
                                                  device_map="auto",
                                                  trust_remote_code=True
                                                 ).eval()

# 加载lora权重
model = PeftModel.from_pretrained(base_model, train_output_dir)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
# 模型推理
# model.eval()
c,_=model.chat(tokenizer, "你是谁？", history=[])
c['name']

'您好，我是 夜行者，由 木子 打造的人工智能助手。请问有什么需要帮助的吗？'